#  Programming Languages (10) --- Rust memory management


Enter your name and student ID.

 * Name:
 * Student ID:


# 1. Introduction
* in this notebook, you are going to learn Rust memory management

# 2. Prepare AI Tutor
* execute the following cell to set up your tutor

In [ ]:
from heytutor import *
config(default_lang="rust")  # choose one of Go/Julia/OCaml/Rust

* you may want to ask a few basics

In [ ]:
I("""borrow-checker basics""")

# 3. Roadmap
# 4. Set up
* check if you can run `rustc` command


In [ ]:
rustc --version

* if it raises an error indicating `rustc` command is not found, execute the following in your shell


In [ ]:
. ~/.cargo/env

# 5. Owning pointer
## 5-1. Assignment of owning pointer does not copy a pointer but moves it
* the following code does _not_ compile
* confirm it compiles if you remove `a.x`


In [ ]:
%%writefile move.rs
#![allow(unused_variables)]

struct S {
    x : i64,
    y : i64,
}

fn main() {
    let a = S{x : 123, y : 456};
    a.x;
    a.y;
    let b = a;
    a.x;
    b.x;
}

In [ ]:
rustc move.rs

## 5-2. Passing argument to a function moves a pointer, too
* the following code does _not_ compile either
* confirm it compiles if you remove `a.x`


In [ ]:
%%writefile move_fun.rs
#![allow(unused_variables)]

struct S {
    x : i64,
    y : i64,
}

fn f(s : S) {
    
}

fn main() {
    let a = S{x : 123, y : 456};
    a.x;
    a.y;
    f(a);
    a.x;
}

In [ ]:
rustc move_fun.rs

## 5-3. `Box::new(e)` is just another way of making an owning pointer
* the following code does _not_ compile
* confirm it compiles if you remove `a.x`


In [ ]:
%%writefile move_box.rs
#![allow(unused_variables)]

struct S {
    x : i64,
    y : i64,
}

fn main() {
    let a = S{x : 123, y : 456};
    a.x;
    a.y;
    let b = Box::new(a);
    a.x;
    b.x;
}

In [ ]:
rustc move_box.rs

## 5-4. Compiler conservatively estimates where owning pointer is alive (OK to dereference)
### 5-4-1. if expression
* the following code does _not_ compile
* if you look closely, we could get that it should be safe, as the condition `a.x == 1234` is always false
* Rust compiler nevertheless rejects this code
  * if either branch of an if expression moves the value out of `a`, the entire if expresson does so


In [ ]:
%%writefile move_if.rs
#![allow(unused_variables)]

struct S {
    x : i64,
    y : i64,
}

fn main() {
    let a = S{x : 123, y : 456};
    a.x;
    a.y;
    if a.x == 1234 {
        let b = a;
    }
    a.x;
}

In [ ]:
rustc move_if.rs

### 5-4-2. loop expression
* the following code does _not_ compile
* again, if you look closely, we could get that it should be safe, as the loop iterates exactly once
* Rust compiler nevertheless rejects this code
  * if an iteration of a loop moves the value out of `a`, `a` becomes invalid at any iteration


In [ ]:
%%writefile move_loop.rs
#![allow(unused_variables)]

struct S {
    x : i64,
    y : i64,
}

fn main() {
    let a = S{x : 123, y : 456};
    a.x;
    a.y;
    for i in 0..1 {
        let b = a;
    }
}

In [ ]:
rustc move_loop.rs

## 5-5. You can never make a cycle solely with owning pointers
* the single-owner rule implies you cannot make a cycle solely with owning pointers
* here is an attempt to make one, which of course does not compile
* it is istructive to see where the compiler complaints


In [ ]:
%%writefile cycle.rs
#![allow(unused_variables)]
#![allow(unused_mut)]

struct S {
    p : Option<Box<S>>
}

fn main() {
    // an attemp to make a cyclic data structure
    // a <-> b
    let mut a = S{p : None};
    let mut b = S{p : None};
    a.p = Some(Box::new(b));
    b.p = Some(Box::new(a));
}

In [ ]:
rustc cycle.rs

# 6. Borrowing pointer
## 6-1. Basics
* the following code _does_ compile
* you can derive a borrowing point by `&e`
* unlike assignment of owning pointer (`let b = a`), an assignment of borrowing pointer does not invalidate the righthand side


In [ ]:
%%writefile borrow.rs
#![allow(unused_variables)]

struct S {
    x : i64,
    y : i64,
}

fn main() {
    let a = S{x : 123, y : 456};
    a.x;
    a.y;
    let b = &a;
    a.x;
    b.x;
}

In [ ]:
rustc borrow.rs

## 6-2. Borrow checking in action
* now the question is how Rust prevents borrowing pointers to already reclaimed data --- data whose owning pointer goes out of scope --- from being dereferenced
* this is a simple example showing it in action


In [ ]:
%%writefile borrow_check.rs
#![allow(unused_variables)]
#![allow(unused_assignments)]
#![allow(dead_code)]

struct S {
    x : i64,
    y : i64,
}

fn main() {
    let c : &S;
    {
        let b : &S;
        let a = S{x : 123, y : 456};
        b = &a;
        c = b;
    }
    c.x;
}

In [ ]:
rustc borrow_check.rs

## 6-3. How borrow-checking actually works?
* Rust tries to associate each borrowing pointer with _the lifetime_ of data it points to (_referent lifetime_)
* roughly speaking,
  * it tries to track from which owning pointer each borrowing pointer is derived from
  * a borrowing pointer can be dereferenced only when the original owning pointer it is derived from is still alive (i.e., within its scope)
* things become complicated due to
  * assignments between borrowing pointers
  * functions taking/returning borrowing pointers
  * structures containing borrowing pointers
* to make this tracking explicit, Rust introduces _lifetime parameters_ to each borrowing pointer
  * `&'a T` ... an immutable borrowing pointer to `T` whose lifetime is `'a`
  * `&mut 'a T` ... a mutable borrowing pointer to `T` whose lifetime is `'a`
* imagine a lifetime parameter such as `'a` represents a _lifetime_ of the data it points to, or more specifically _the set of program points where the data it points to is still alive_

## 6-4. Functions taking borrowing pointers
* you have to attach lifetime parameters to each parameter having a borrowing pointer type (`&T`)

* the following program does _not_ compile, because lifetime parameters are lacking
* practice: annotate it with lifetime parameters


In [ ]:
%%writefile borrow_fun.rs
#![allow(unused_variables)]
#![allow(unused_assignments)]
#![allow(dead_code)]
#![allow(unused_must_use)]

/*
(1) need to add lifetime parameters to function parameter types
(2) the function must take those lifetime parameters
 */
fn foo(ra : &i32, rb : &i32, rc : &i32) -> &i32 {
    ra
}

fn main() {
    let r : &i32;
    let a = 123;
    {
        let b = 456;
        {
            let c = 789;
            r = foo(&a, &b, &c);
        }
    }
    *r;
}

In [ ]:
rustc borrow_fun.rs

## 6-5. Structures having borrowing pointers
* you have to attach lifetime parameters to each field having a borrowing pointer type (e.g., `&T` -> `&'a T`)
* the resulting structure has to take these parameters (e.g., `S` -> `S<'a,'b,...>`)
* the following program does _not_ compile, because lifetime parameters are lacking
* practice: annotate it with lifetime parameters
* even after fixing them by attaching lifetime parameters, it still does not compile because dereference `a.b.c.x` is unsafe
* confirm that it compiles if you remove either `a.b.c.x` or replace the assignment `a.b = &b_` to `a.b = &b`


In [ ]:
%%writefile borrow_data.rs
#![allow(unused_variables)]
#![allow(unused_mut)]

/*
(1) need to add lifetime parameters to reference types
(2) struct must take lifetime parameters that appear in fields
 */

struct A { b : &B }
struct B { c : &C }
struct C { x : i32 }

fn main() {
    let c = C{x : 123};
    let b = B{c : &c};
    let mut a = A{b : &b};
    {
        let b_ = B{c : &c};
        a.b = &b_;
    }
    a.b.c.x;                    // OK?
}

In [ ]:
rustc borrow_data.rs

## 6-6. An example using functions and data structures
* practice: annotate the following program with lifetime parameters
* even after fixing them by attaching lifetime parameters, it still does not compile because dereference `a.b.c.x` is unsafe
* confirm that it compiles if you remove either `a.b.c.x` or replace the function call `baz(&mut a, &b_)` to `baz(&mut a, &b)`


In [ ]:
%%writefile borrow_data_fun.rs
#![allow(unused_variables)]
#![allow(unused_mut)]

/*
(1) need to add lifetime parameters to reference types
(2) struct must take lifetime parameters that appear in fields
 */

struct A { b : &B }
struct B { c : &C }
struct C { x : i32 }

fn baz(a : &mut A, b : &B {
    a.b = b
}

fn main() {
    let c = C{x : 123};
    let b = B{c : &c};
    let mut a = A{b : &b};
    {
        let b_ = B{c : &c};
        baz(&mut a, &b_);
    }
    a.b.c.x;                    // OK?
}

In [ ]:
rustc borrow_data_fun.rs

## 6-7. The exact contraint imposed on dereferencing data structure
* the goal is to prevent borrowing pointers to reclaimed data from being dereferenced
* that is, if a borrowing pointer has type `&'a T`, prevent it from being dereferenced at program points not contained in lifetime represented by `'a`
* the actual rule is slightly more strict
* Rust disallows dereferencing any data structure having lifetime parameters `'a, 'b, 'c, ...` at any program point outside any of them
* that is, dereferencing a struct of type `S<'a,'b,'c,...>` is allowed only at program points contained in all of `'a,'b,'c,...`

* the following program is actually _safe_ if you closely observe that at (*), `s.b` is a dangling pointer but `s.a` is not
* this program nevertheless does not compile, as type of `s` (S<'a,'b>) has lifetime parameters `'a` and `'b` and
  * `'b` represents a lifetime not containing the program point (*), due to the assignment `s.b = &b;`


In [ ]:
%%writefile partial_dangling.rs
#![allow(dead_code)]
#![allow(unused_mut)]
#![allow(unused_must_use)]
#![allow(unused_variables)]
  
struct S<'a,'b> {
    a : &'a i32,
    b : &'b i32
}

fn main() {
    let a = 123;
    let mut s = S{a: &a, b: &a};
    {
        let b = 456;
        s.b = &b;
    }
    *s.a; // (*)
}

In [ ]:
rustc partial_dangling.rs